In [1]:
import numpy as np
import ndio.remote.neurodata as neurodata
from time import time

startTime = time()

nd = neurodata()

token = 'kasthuri2015_ramon_v4'
channel = 'synapses'
res = 3
pixel_dim = 0.024*0.024*0.030  #can get from LIMS

In [2]:
'''
There were 1,700 synapses at a density of one synapse per 1.13 μm3.

'''

import ndio.ramon as ramon

synapse_ids = nd.get_ramon_ids(token, channel, ramon_type=ramon.RAMONSynapse)

# A priori known bounds for cylinders.  Alternatively we could sweep over entire volume - this is more efficient.
# TODO:  assume that all synapses are inside cylinders, which we know to be true - should do with manual masking or a 
# RAMONId predicate query

xbox = [694,1794]; 
ybox = [1750, 2460];
zbox = [1004, 1379];

# These calls take about 60 seconds to execute
gcyl = nd.get_volume('kat11greencylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
rcyl = nd.get_volume('kat11redcylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
bcyl = nd.get_volume('kat11mojocylinder','annotation', xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)


mask = (gcyl.cutout + rcyl.cutout + bcyl.cutout) > 0

mask_pixels = sum(sum(sum(mask)))
mask_volume = mask_pixels * pixel_dim
print 'Mask Pixels: ' + str(mask_pixels)
print 'Mask Volume: ' + str(mask_volume) + ' um^3'

syn = nd.get_volume(token,channel, xbox[0], xbox[1], ybox[0], ybox[1], zbox[0], zbox[1], resolution = res)
syn_volume = np.count_nonzero(syn.cutout)

print 'density of synapses is: {} / um^3'.format(round(1.0*(np.shape(synapse_ids)[0]) / mask_volume,4))
print 'fraction of volume that is psd: ' + str(round(1.0*syn_volume/mask_pixels,4))

print time() - startTime

Mask Pixels: 90152653
Mask Volume: 1557.83784384 um^3
density of synapses is: 1.0913 / um^3
fraction of volume that is psd: 0.0089
430.658550978


In [3]:
# Retrieve metadata for all objects
import ndio.ramon as ramon

synapse_ids = nd.get_ramon_ids(token, channel, ramon_type=ramon.RAMONSynapse)

# Get all synapses
synAll = nd.get_ramon(token,channel,synapse_ids)
print '\ntime elapsed so far: ' + str(time()-startTime)



time elapsed so far: 586.225799799


In [4]:
vars(synAll[5]) 

{'author': u'unspecified',
 'confidence': 1.0,
 'cutout': None,
 'id': u'21',
 'kvpairs': {u'axonSynapseType': u'en-passant',
  u'multiSynapseBouton': u'unknown',
  u'postsynaptic': u'3234',
  u'presynaptic': u'3692',
  u'psdSize': u'3588',
  u'synapseLocation': u'spine',
  u'vesicleCount': u'985'},
 'resolution': 0,
 'segments': [[3234, 2], [3692, 1]],
 'status': 0,
 'synapse_type': 0,
 'voxels': None,
 'weight': 0.0,
 'xyz_offset': (0, 0, 0)}